In [6]:
!pip install selenium pandas yagmail webdriver-manager


## Import The Necessary Libraries

In [14]:
import yagmail
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import time


In [27]:

# Function to initialize the Chrome WebDriver
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")  # Open Chrome maximized
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# Function to scrape job details and return structured data (list of dictionaries)
def scrape_job_details(driver, url):
    driver.get(url)
    time.sleep(3)  # Wait for the page to load
    
    job_details = []
    
    # Find job cards on the page
    job_cards = driver.find_elements(By.CSS_SELECTOR, ".job-info")
    
    for card in job_cards:
        try:
            # Extract the job title and URL
            title_element = card.find_element(By.CSS_SELECTOR, ".job-list-li a")
            job_title = title_element.text
            job_url = title_element.get_attribute("href")
            
            # Extract the job date
            date_posted_element = card.find_element(By.CSS_SELECTOR, "[class='job-item']")
            date_posted = date_posted_element.text
            
            # Append job details as a dictionary to the list
            job_details.append({
                'Job Title': job_title,
                'Job URL': job_url,
                'Date Posted': date_posted
            })
        except Exception as e:
            print(f"Error extracting job details: {e}")
    
    return job_details

# Function to save job details to a CSV file
def save_to_csv(data, filename):
    # Convert list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(data)
    # Save DataFrame to CSV
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

# Function to send an email with the scraped job details
def send_email(subject, body, to_email):
    try:
        # Initialize yagmail SMTP client with your email address
        yag = yagmail.SMTP('wangechieunice0@gmail.com')
        
        # Send email
        yag.send(
            to=to_email,
            subject=subject,
            contents=body
        )
        print("Email sent successfully")
    except Exception as e:
        print(f"Error sending email: {e}")

# Main function to automate scraping, saving to CSV, and email
def main():
    driver = setup_driver()
    
    try:
        # Example usage
        search_url = "https://www.myjobmag.co.ke/search/jobs?location=Nairobi&field=Data%2C+Business+Analysis+and+AI&education=BA%2FBSc%2FHND&experience=1+-+4"
        job_details = scrape_job_details(driver, search_url)

        # Save job details to a CSV file
        csv_filename = "nairobi_jobs_data_analysis.csv"
        save_to_csv(job_details, csv_filename)

        # Email settings
        subject = "Nairobi Job Listings"
        to_email = 'wangechieunice0@gmail.com'  # Email address to send to

        # Prepare email body with job details
        job_details_str = "\n".join([f"Job Title: {job['Job Title']}\nJob URL: {job['Job URL']}\nDate Posted: {job['Date Posted']}\n" + "-" * 40 for job in job_details])
        
        # Send the email with the job details
        send_email(subject, job_details_str, to_email)
    
    finally:
        # Close the browser
        driver.quit()

if __name__ == "__main__":
    main()


Data saved to nairobi_jobs_data_analysis.csv
Email sent successfully


In [28]:
df = pd.read_csv("nairobi_jobs_data_analysis.csv")
df.head()

,Job Title,Job URL,Date Posted
0,Partner Mobilization and Data Consultant at UN...,https://www.myjobmag.co.ke/job/partner-mobiliz...,19 September
1,Data Analyst at Kenya National Chamber of Comm...,https://www.myjobmag.co.ke/job/data-analyst-ke...,17 September
2,Data QA/QC Officer - Kenya at BURN,https://www.myjobmag.co.ke/job/data-qa-qc-offi...,17 September
3,Document & Data Operations Associate - Fixed T...,https://www.myjobmag.co.ke/job/document-data-o...,16 September
4,MLE Field Officer-Temporary at Evidence Action,https://www.myjobmag.co.ke/job/mle-field-offic...,16 September


In [29]:
df.tail()

,Job Title,Job URL,Date Posted
13,Power BI Developer at Stratostaff,https://www.myjobmag.co.ke/job/power-bi-develo...,05 September
14,"Monitoring, Evaluation and Learning Officer at...",https://www.myjobmag.co.ke/job/monitoring-eval...,05 September
15,Amref Ethics and Scientific Intern at Amref Kenya,https://www.myjobmag.co.ke/job/amref-ethics-an...,03 September
16,Data Security Engineer at Tezza Business Solut...,https://www.myjobmag.co.ke/job/data-security-e...,30 August
17,Spatial Data Analyst at UN-Habitat,https://www.myjobmag.co.ke/job/spatial-data-an...,29 August
